In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()


# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Explore the data
inspector = inspect(engine)
columns = inspector.get_columns('measurement')

for column in columns:
    print(column["name"], column["type"])

In [ ]:
inspector = inspect(engine)
columns = inspector.get_columns('station')

for column in columns:
    print(column["name"], column["type"])

In [ ]:
# Understand the data first

# engine.execute('select * from measurement limit 5').fetchall()
engine.execute('select * from station limit 5').fetchall()

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
most_recent_date = session.query(func.max(Measurement.date))
most_recent_date[0][0]

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
to_date = datetime.strptime(most_recent_date[0][0], '%Y-%m-%d')
to_date

In [ ]:
# Calculate the date one year from the last date in data set.

from_date = to_date - relativedelta(months=+12)
from_date = from_date.date()
to_date = to_date.date()

In [ ]:
# Perform a query to retrieve the data and precipitation scores
stmt = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date > from_date).\
                filter(Measurement.date < to_date).statement

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
one_year_df = pd.read_sql_query(stmt, session.bind)
one_year_df.head()

In [ ]:
# Check that the data is right
# one_year_df['date'].min()
one_year_df['date'].max()

In [ ]:
# Sort the dataframe by date
one_year_df['date'] = pd.to_datetime(one_year_df['date'])
one_year_df = one_year_df.sort_values(by='date')
one_year_df = one_year_df.dropna(how='any')
one_year_df.head()

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
date_x = one_year_df['date']
precipitation_y = one_year_df['prcp']

plt.title("Precipitation Data - Last 12 Months")
plt.xlabel("Date")
plt.ylabel("Precipitation (inches)")
plt.plot(date_x, precipitation_y, color="blue", linewidth=1)
plt.xticks(rotation=90)
plt.show()


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
one_year_df.describe()


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
station_count = session.query(Station.station).count()
station_count

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.

most_active_stations = session.query(Measurement.station, func.count(Measurement.id)).\
                        group_by(Measurement.station).order_by(func.count(Measurement.id).desc()).all()

for station in most_active_stations:
    print(station)

In [ ]:
# Check the Measurement table for list of unique stations with measurement data

query = session.query(Measurement.station.distinct().label("title"))
titles = [row.title for row in query.all()]
titles

In [ ]:
# Check the Station table for list of unique stations with measurement data

query = session.query(Station.name.distinct().label("title"))
titles = [row.title for row in query.all()]
titles

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
# Get the station with the most measurements

statement = session.query(Measurement.station, func.count(Measurement.id)).\
                        group_by(Measurement.station).order_by(func.count(Measurement.id).desc()).limit(1).statement
station_df = pd.read_sql_query(statement, session.bind)
most_active_station_id = station_df['station'][0]

In [ ]:
# Use the station id to get temp data

results = session.query(func.max(Measurement.tobs), func.min(Measurement.tobs),func.avg(Measurement.tobs)).\
            filter(Measurement.station == most_active_station_id)
for result in results:
    print(f"The highest temperature is {result[0]}, the lowest is {result[1]} and the average is {result[2]}")

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram

stmt = session.query(Measurement.date, Measurement.tobs).filter(Measurement.date > from_date).\
                filter(Measurement.date < to_date).filter(Measurement.station == most_active_station_id).statement

top_station_one_year_df = pd.read_sql_query(stmt, session.bind)
top_station_one_year_df.head()

In [ ]:
tobs = top_station_one_year_df['tobs']

plt.hist(tobs, 20, density=True, alpha=0.5, label="Temperature")
plt.show()

# Close session

In [ ]:
# Close Session
session.close()